In [1]:
import Utils
from DataLoader import DataLoader
from FeatureCreator import FeatureCreator
from Model import Model
from CouponCreator import CouponCreator

In [5]:
config = Utils.read_json('../config.json')
Utils.print_json(config)

{
    "input": {
        "path": "../input/",
        "files": {
            "baskets": "baskets.parquet",
            "coupons": "coupons.parquet",
            "coupons_index": "coupon_index.parquet"
        }
    },
    "model": {
        "test_week": 90,
        "n_shoppers": 2000,
        "n_products": 250,
        "train_window": 4,
        "trend_windows": [
            1,
            3,
            5
        ],
        "n_coupons": 5,
        "discounts": [
            0.15,
            0.2,
            0.25,
            0.3
        ]
    },
    "output": {
        "path": "../output/"
    }
}


In [6]:
# DATA LOADING
dataloader = DataLoader(config)
dataset = dataloader.get_dataset()

Read dataset.parquet.gzip from cache...
dataset.parquet.gzip was not found in cache.
Executing callback function "create_dataset" to create dataset.parquet.gzip
57.5% memory reduction for baskets.parquet (from 2626.10 MB to 1116.09 MB)
60.0% memory reduction for coupons.parquet (from 1716.61 MB to 686.65 MB)
62.5% memory reduction for coupon_index.parquet (from 0.31 MB to 0.11 MB)
replaced missing prices with mode
Wrote dataset.parquet.gzip to ../cache/
Successfully read dataset.parquet.gzip into memory.


In [7]:
dataset

,week,shopper,product,price,discount,purchased,week_hist,discount_received_weeks,discount_redeemed_weeks
0,86,0,0,688.0,0.0,0.0,NaN,NaN,NaN
1,86,0,1,560.0,0.0,0.0,NaN,"[57, 61]",NaN
2,86,0,2,773.0,0.0,0.0,NaN,"[44, 45, 88]",NaN
3,86,0,3,722.0,0.0,0.0,NaN,"[22, 87]",NaN
4,86,0,4,620.0,0.0,0.0,"[24, 40, 45, 51, 54, 58, 60, 70]","[12, 17, 38]",NaN
...,...,...,...,...,...,...,...,...,...
2499995,90,1999,245,549.0,0.0,0.0,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89]","[5, 22, 34, 85]","[5, 22]"
2499996,90,1999,246,702.0,0.0,0.0,NaN,"[30, 83, 89]",NaN
2499997,90,1999,247,670.0,0.0,0.0,NaN,"[28, 56, 59]",NaN
2499998,90,1999,248,490.0,0.0,0.0,NaN,"[57, 68]",NaN


In [8]:
# FEATURE CREATION
feature_creator = FeatureCreator(dataset, config)
model_data = feature_creator.get_model_data()
model_data

Read model_data.parquet.gzip from cache...
model_data.parquet.gzip was not found in cache.
Executing callback function "create_features" to create model_data.parquet.gzip
Wrote model_data.parquet.gzip to ../cache/
Successfully read model_data.parquet.gzip into memory.


,week,shopper,product,price,discount,purchased,product_cat,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_product_redemption_rate,discount_buy,shopper_redemption_rate,product_redemption_rate
0,86,0,0,688.0,0.0,0.0,0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
500000,87,0,0,688.0,0.0,0.0,0,0.15,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
1000000,88,0,0,688.0,0.0,0.0,0,0.20,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
1500000,89,0,0,688.0,0.0,0.0,0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
2000000,90,0,0,688.0,0.0,0.0,0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499999,86,1999,249,499.0,0.0,0.0,24,0.10,0,0,1,31.0,0.0,0.0,0.0,0.093023,0.25,0.5,0.095556,0.128790
999999,87,1999,249,499.0,0.0,0.0,24,0.00,0,0,1,32.0,0.0,0.0,0.0,0.091954,0.25,0.5,0.095556,0.128790
1499999,88,1999,249,499.0,0.0,0.0,24,0.00,0,0,1,33.0,0.0,0.0,0.0,0.090909,0.25,0.5,0.095556,0.128790
1999999,89,1999,249,499.0,0.0,0.0,24,0.30,0,0,1,34.0,0.0,0.0,0.0,0.089888,0.25,0.5,0.095556,0.128790


In [9]:
model_data

,week,shopper,product,price,discount,purchased,product_cat,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_product_redemption_rate,discount_buy,shopper_redemption_rate,product_redemption_rate
0,86,0,0,688.0,0.0,0.0,0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
500000,87,0,0,688.0,0.0,0.0,0,0.15,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
1000000,88,0,0,688.0,0.0,0.0,0,0.20,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
1500000,89,0,0,688.0,0.0,0.0,0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
2000000,90,0,0,688.0,0.0,0.0,0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.055556,0.040919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499999,86,1999,249,499.0,0.0,0.0,24,0.10,0,0,1,31.0,0.0,0.0,0.0,0.093023,0.25,0.5,0.095556,0.128790
999999,87,1999,249,499.0,0.0,0.0,24,0.00,0,0,1,32.0,0.0,0.0,0.0,0.091954,0.25,0.5,0.095556,0.128790
1499999,88,1999,249,499.0,0.0,0.0,24,0.00,0,0,1,33.0,0.0,0.0,0.0,0.090909,0.25,0.5,0.095556,0.128790
1999999,89,1999,249,499.0,0.0,0.0,24,0.30,0,0,1,34.0,0.0,0.0,0.0,0.089888,0.25,0.5,0.095556,0.128790


In [10]:
# MODELING - Train-Test-Split
model = Model(model_data)
X_train, y_train, X_test, y_test = model.train_test_split(config)

/Users/sascha/Desktop/mlim-g2/env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/sascha/Desktop/mlim-g2/env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [11]:
X_train

,price,discount,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_product_redemption_rate,discount_buy,shopper_redemption_rate,product_redemption_rate,shopper_WOE,product_WOE
0,688.0,0.0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.055556,0.040919,0.074996,-0.595444
500000,688.0,0.0,0.15,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.055556,0.040919,0.074996,-0.595444
1000000,688.0,0.0,0.20,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.055556,0.040919,0.074996,-0.595444
1500000,688.0,0.0,0.00,0,1,0,104.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.055556,0.040919,0.074996,-0.595444
250,688.0,0.0,0.00,0,1,0,5.0,0.0,0.0,0.2,0.069767,0.00,0.166667,0.057778,0.040919,0.105881,-0.595444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999749,499.0,0.0,0.40,0,0,1,104.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.035556,0.128790,-0.491417,0.761272
499999,499.0,0.0,0.10,0,0,1,31.0,0.0,0.0,0.0,0.093023,0.25,0.500000,0.095556,0.128790,0.248261,0.761272
999999,499.0,0.0,0.00,0,0,1,32.0,0.0,0.0,0.0,0.091954,0.25,0.500000,0.095556,0.128790,0.248261,0.761272
1499999,499.0,0.0,0.00,0,0,1,33.0,0.0,0.0,0.0,0.090909,0.25,0.500000,0.095556,0.128790,0.248261,0.761272


In [9]:
X_test

,price,discount,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_product_redemption_rate,discount_buy,shopper_redemption_rate,product_redemption_rate,shopper_WOE,product_WOE
2000000,688.0,0.0,0.0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.055556,0.040919,0.039796,-0.541070
2000250,688.0,0.0,0.0,0,1,0,8.0,0.0,0.000000,0.0,0.067416,0.000000,0.166667,0.057778,0.040919,0.039796,-0.541070
2000500,688.0,0.0,0.0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.037778,0.040919,-0.392694,-0.541070
2000750,688.0,0.0,0.0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.062222,0.040919,0.007017,-0.541070
2001000,688.0,0.0,0.0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.040000,0.040919,-0.345154,-0.541070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498999,499.0,0.0,0.0,0,0,1,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.062222,0.128790,0.039796,0.770442
2499249,499.0,0.0,0.0,0,0,1,1.0,1.0,0.333333,0.4,0.483146,1.000000,0.069767,0.064444,0.128790,0.244864,0.770442
2499499,499.0,0.0,0.0,0,0,1,3.0,0.0,0.333333,0.2,0.157303,0.666667,0.214286,0.048889,0.128790,-0.392694,0.770442
2499749,499.0,0.0,0.4,0,0,1,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.035556,0.128790,-0.256100,0.770442


In [12]:
# FITTING THE MODEL
model.fit(X_train, y_train)
y_hat = model.predict(X_train)
log_loss_score = model.log_loss_score(y_train, y_hat)
log_loss_score

0.07585938672134537

In [11]:
y_hat = model.predict(X_test)
model.log_loss_score(y_test, y_hat)

# we have leakage i guess --> redemption features --> no leakage any more if we build redemption features on data before the train data --> not the best way

0.0768575724650346

In [12]:
coupon_creator = CouponCreator(model)

In [13]:
X_template = X_test.copy()
X_template['discount'] = None
X_template['substitue_discount'] = 0
X_template

,price,discount,substitue_discount,cluster_0,cluster_1,cluster_2,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_product_redemption_rate,discount_buy,shopper_redemption_rate,product_redemption_rate,shopper_WOE,product_WOE
2000000,688.0,None,0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.055556,0.040919,0.039796,-0.541070
2000250,688.0,None,0,0,1,0,8.0,0.0,0.000000,0.0,0.067416,0.000000,0.166667,0.057778,0.040919,0.039796,-0.541070
2000500,688.0,None,0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.037778,0.040919,-0.392694,-0.541070
2000750,688.0,None,0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.062222,0.040919,0.007017,-0.541070
2001000,688.0,None,0,0,1,0,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.040000,0.040919,-0.345154,-0.541070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498999,499.0,None,0,0,0,1,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.062222,0.128790,0.039796,0.770442
2499249,499.0,None,0,0,0,1,1.0,1.0,0.333333,0.4,0.483146,1.000000,0.069767,0.064444,0.128790,0.244864,0.770442
2499499,499.0,None,0,0,0,1,3.0,0.0,0.333333,0.2,0.157303,0.666667,0.214286,0.048889,0.128790,-0.392694,0.770442
2499749,499.0,None,0,0,0,1,103.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.035556,0.128790,-0.256100,0.770442


In [20]:
discounts = config['model']['discounts']

In [15]:
revenue = coupon_creator.get_revenue(X_template, config['model']['discounts'])
revenue

,week,shopper,product,price,product_cat,discount,probabilities,exp_revenue,delta_revenue
2000000,89,0,0,688.0,0,0.0,0.001189,0.818212,None
2000250,89,1,0,688.0,0,0.0,0.065378,44.980143,None
2000500,89,2,0,688.0,0,0.0,0.000963,0.662256,None
2000750,89,3,0,688.0,0,0.0,0.001189,0.818212,None
2001000,89,4,0,688.0,0,0.0,0.001159,0.797074,None
...,...,...,...,...,...,...,...,...,...
2498999,89,1995,249,499.0,24,0.3,0.005008,1.749158,1.000671
2499249,89,1996,249,499.0,24,0.3,0.745729,260.482975,10.215381
2499499,89,1997,249,499.0,24,0.3,0.322171,112.534185,41.523463
2499749,89,1998,249,499.0,24,0.3,0.003746,1.308304,0.692618


In [17]:
coupon_creator.create_top_coupons(revenue, 5)

,shopper,week,coupon,product,discount,product_cat
2000067,0,89,0,67,0.15,6
2000171,0,89,1,171,0.15,17
2000157,0,89,2,157,0.15,15
2000116,0,89,3,116,0.15,11
2000076,0,89,4,76,0.30,7
...,...,...,...,...,...,...
2062339,249,89,0,89,0.30,8
2062363,249,89,1,113,0.20,11
2062262,249,89,2,12,0.20,1
2062313,249,89,3,63,0.15,6


In [ ]:
coupons = coupon_creator.get_top_coupons(discounts, X_template)
coupons